In [ ]:
import boto3
ec2 = boto3.resource('ec2')
elbv2 = boto3.client('elbv2')
asg = boto3.client('autoscaling')
route53 = boto3.client('route53')

In [ ]:
# Let's keep all the names in one place
load_balancer_name = 'application-load-balancer'
instances_ami_name = 'java-application-ami'
instances_security_group_name = 'instances-security-group'
balancer_security_group_name = 'balancer-security-group'
launch_configuration_name = 'asg-launch-configuration'
target_group_name = 'asg-target-group'
auto_scaling_group_name = 'auto-scaling-group'
domain_name = 'doug-nicholson.net'
balancer_host_name = 'asg.' + domain_name

In [ ]:
# add the public ip address to route53 so SSH is easier
response = route53.list_hosted_zones_by_name(
    DNSName=domain_name
)
local_zone_id = response['HostedZones'][0]['Id'][12:]
response = route53.list_resource_record_sets(
    HostedZoneId=local_zone_id,
    StartRecordName=balancer_host_name,
    StartRecordType='A',
    MaxItems='1'
)
for record_set in response['ResourceRecordSets']:
    if record_set['Name'] == balancer_host_name + '.':
        response = route53.change_resource_record_sets(
            HostedZoneId=local_zone_id,
            ChangeBatch={
                'Changes': [
                    {
                        'Action': 'DELETE',
                        'ResourceRecordSet': record_set
                    }
                ]
            }
        )
        # it seems like waiting will be the right thing to do
        route53_waiter = route53.get_waiter('resource_record_sets_changed')
        route53_waiter.wait(
            Id=response['ChangeInfo']['Id'][8:]
        )
        print('record for ' + balancer_host_name + ' deleted')

In [ ]:
# Delete the auto-scaling group by force because updating the auto-scaling
# group to cause the termination of the instances takes a long time.
try:
    response = asg.describe_auto_scaling_groups(
        AutoScalingGroupNames=[
            auto_scaling_group_name
        ]
    )
except:
    pass
else:
    response = asg.delete_auto_scaling_group(
        AutoScalingGroupName=auto_scaling_group_name,
        ForceDelete=True
    )

In [ ]:
# Delete the application load balancer
try:
    response = elbv2.describe_load_balancers(
        Names=[
            load_balancer_name
        ]
    )
except:
    pass
else:
    for load_balancer in response['LoadBalancers']:
        load_balancer_arn = load_balancer['LoadBalancerArn']
        response = elbv2.delete_load_balancer(
            LoadBalancerArn=load_balancer_arn
        )
        print('load balancer deleted')

In [ ]:
# Wait until the instances terminate
instance_iterator = ec2.instances.filter(
    Filters=[
        {
            'Name': 'instance.group-name',
            'Values': [
                instances_security_group_name
            ]
        }
    ]
)
for instance in instance_iterator:
    instance.terminate()
for instance in instance_iterator:
    instance.wait_until_terminated()
print('instances terminated')

In [ ]:
# Delete target group
try:
    response = elbv2.describe_target_groups(
        Names=[
            target_group_name
        ]
    )
except:
    pass
else:
    target_group_arn = response['TargetGroups'][0]['TargetGroupArn']
    response = elbv2.delete_target_group(
        TargetGroupArn=target_group_arn
    )
    print('target group deleted')

In [ ]:
# Delete the launch configuration
response = asg.describe_launch_configurations(
    LaunchConfigurationNames=[
        launch_configuration_name
    ]
)
for launch_configuration in response['LaunchConfigurations']:
    if launch_configuration['LaunchConfigurationName'] == launch_configuration_name:
        response = asg.delete_launch_configuration(
            LaunchConfigurationName=launch_configuration_name
        )

In [ ]:
# Delete the security groups
security_group_names = [instances_security_group_name, balancer_security_group_name]
security_group_iterator = ec2.security_groups.filter(
    Filters=[
        {
            'Name': 'group-name',
            'Values': security_group_names
        }
    ]
)
# We can't delete security groups that reference other security
# group so revoke the ingress rules of all security groups first.
for security_group in security_group_iterator:
    response = security_group.revoke_ingress(
        IpPermissions=security_group.ip_permissions
    )
for security_group in security_group_iterator:
    security_group.reload()
    response = security_group.delete()
print('security groups deleted')


In [ ]:
# Success!
print('cleanup auto-scaling group script completed')